# ML for breathing

Applying simple, common machine learning models to segments of EMG to predict whether segment was with our without respiratory muscle effort.

## Import libraries

In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
import collections
from collections import namedtuple
import builtins
import math
from scipy.stats import entropy

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree

## Import and set up data ( see ML_snippet_maker notebook for generation)

In [3]:

csv = pd.read_csv('../researcher_interface/ML_files/for_ml_csv.csv')
csv2 = pd.read_csv('../researcher_interface/ML_files/for_ml_csv2.csv')

In [4]:
csv2.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,991,992,993,994,995,996,997,998,999,label
0,0,0.000059,0.000055,0.000044,0.000039,0.000170,0.000218,0.000110,0.000061,0.000186,...,0.000021,0.000042,5.928041e-05,0.000069,0.000038,0.000040,0.000102,1.012105e-04,0.000054,exhale
1,1,0.000184,0.000016,0.000167,0.000237,0.000167,0.000027,0.000084,0.000080,0.000046,...,0.000036,0.000035,6.349561e-05,0.000040,0.000021,0.000079,0.000079,2.753498e-05,0.000024,exhale
2,2,0.000040,0.000155,0.000183,0.000081,0.000122,0.000079,0.000111,0.000013,0.000076,...,0.000104,0.000251,2.212753e-04,0.000087,0.000023,0.000048,0.000020,2.833529e-07,0.000018,exhale
3,3,0.000130,0.000068,0.000023,0.000010,0.000072,0.000043,0.000043,0.000044,0.000068,...,0.000092,0.000077,4.719800e-08,0.000100,0.000139,0.000062,0.000064,1.051652e-04,0.000033,exhale
4,4,0.000212,0.000112,0.000047,0.000122,0.000386,0.000544,0.000385,0.000042,0.000408,...,0.000039,0.000111,7.080898e-05,0.000034,0.000106,0.000104,0.000042,3.894382e-05,0.000074,exhale


In [5]:
csv = csv.dropna()
csv2 = csv2.dropna()

In [6]:
listicle = (list(range(999)))
fu = list(map(str, listicle))

In [7]:
csv2['std'] = csv2.std(axis=1)
csv2['max'] = csv2.max(axis=1)
csv2['min'] = csv2.min(axis=1)
csv2['mean'] = csv2.mean(axis=1)
csv2['entropy'] = csv2[fu].apply(entropy, axis =1)


C:\Users\makeda\AppData\Local\Temp\ipykernel_29956\3937976712.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  csv2['std'] = csv2.std(axis=1)
C:\Users\makeda\AppData\Local\Temp\ipykernel_29956\3937976712.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  csv2['max'] = csv2.max(axis=1)
C:\Users\makeda\AppData\Local\Temp\ipykernel_29956\3937976712.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  csv2['min'] = csv2.min(axis=1)
C:\Users\makeda\AppData\Local\Temp\ipykernel_29956\3937

In [8]:
csv['std'] = csv.std(axis=1)
csv['max'] = csv.max(axis=1)
csv['min'] = csv.min(axis=1)
csv['mean'] = csv.mean(axis=1)
csv['entropy'] = csv[fu].apply(entropy, axis =1)

C:\Users\makeda\AppData\Local\Temp\ipykernel_29956\65018474.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  csv['std'] = csv.std(axis=1)
C:\Users\makeda\AppData\Local\Temp\ipykernel_29956\65018474.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  csv['max'] = csv.max(axis=1)
C:\Users\makeda\AppData\Local\Temp\ipykernel_29956\65018474.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  csv['min'] = csv.min(axis=1)
C:\Users\makeda\AppData\Local\Temp\ipykernel_29956\65018474.py:4: F

In [9]:
csv.loc[csv['label']== 'exhale', 'label'] = 0
csv.loc[csv['label']== 'inhale', 'label'] = 1
csv2.loc[csv2['label']== 'exhale', 'label'] = 0
csv2.loc[csv2['label']== 'inhale', 'label'] = 1

In [10]:
csv_to = pd.concat([csv,csv2])

## ML

In [11]:
X = csv_to.drop('label', axis =1)
X = X[['std','max', 'min', 'mean', 'entropy']].values
X = X.astype('float')


In [12]:
y = csv_to['label'].values
y=y.astype('int')


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# scale
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [15]:
svm = SVC(kernel= 'linear', random_state=1, C=0.1)
svm.fit(X_train, y_train)

SVC(C=0.1, kernel='linear', random_state=1)

In [16]:
y_pred = svm.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

Accuracy: 0.952


In [17]:
lr = LogisticRegression(solver='liblinear', random_state=0)
lr.fit(X_train, y_train)

LogisticRegression(random_state=0, solver='liblinear')

In [18]:
y_pred = lr.predict(X_test)

In [19]:
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

Accuracy: 0.952


In [20]:
dt = tree.DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [21]:
y_pred = dt.predict(X_test)

In [22]:
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

Accuracy: 0.952


OK, none of these models worked so well until adding entropy, then they worked 100%. but remember we only had a few features in our dataset, of about 100 samples. What we can do it add 3X on data and features, with and without entropy. Will discuss with scientists exactly what features they want.